A* implementation

In [0]:
import numpy as np
import pandas as pd
from datetime import datetime

class State:
  def __init__(self, parent=None, position=None):
    self.parent = parent
    self.position = position

    self.g = 0
    self.f = 0
    self.h = 0
  
  def __eq__(self, other):
    return self.position == other.position

def retrieve_path(currNode, maze):
  path = []
  no_rows, no_colmns = np.shape(maze)
  result = [[-1 for i in range(no_colmns)]for j in range(no_rows)]
  current = currNode
  while current is not None:
    path.append(current.position)
    current = current.parent
  path = path[::-1]
  start = 0
  for i in range(len(path)):
    result[path[i][0]][path[i][1]] = start
    start += 1
  return result

def maze_solve(maze, cost, start, end):
  initialState = State(None, tuple(start))
  initialState.g = initialState.f = initialState.h = 0
  finalState = State(None, tuple(end))
  finalState.g = finalState.h = finalState.h =0

  toVisitList = []
  visitedList = []

  toVisitList.append(initialState)

  out_inter = 0
  max_iter = (len(maze)//2)**10

  move = [[-1,0], #Up
          [0,-1], #Left
          [1,0],  #Down
          [0,1]]  #Right
  
  no_rows, no_colmns = maze.shape[0], maze.shape[1]  #np.shape(maze)#
  count = 0

  while len(toVisitList) > 0:
    out_inter += 1

    currState = toVisitList[0]
    currIndex = 0
    for index, item in enumerate(toVisitList):
      if item.f < currState.f:
        currState = item
        currIndex = index

    if out_inter > max_iter:
      print("Taking too long to find path!!")
      return (currState, maze)

    toVisitList.pop(currIndex)
    visitedList.append(currState)

    count+=1

    if currState == finalState:
      return count #retrieve_path(currState, maze)

    children = []

    for newState in move:
      statePosition = (currState.position[0] + newState[0], currState.position[1] + newState[1]) 

      if (statePosition[0] > (no_rows-1) or statePosition[0] < 0 or statePosition[1] > (no_colmns-1) or statePosition[1] < 0):
        continue
      
      if maze[statePosition[0]][statePosition[1]] != 0:
        continue

      newState = State(currState, statePosition)
      children.append(newState)

    for child in children:

      if len([visitedChild for visitedChild in visitedList if visitedChild == child]) > 0:
        continue
        
      child.g = currState.g + cost
      child.h = (((child.position[0] - finalState.position[0])**2)+((child.position[1] - finalState.position[1])**2))

      child.f = child.g + child.h

      if len([i for i in toVisitList if child == i and child.g > i.g]) > 0:
        continue

      toVisitList.append(child)

if __name__ == '__main__':

  # maze = [[0,1,0,0,0,0],
  #         [0,0,0,0,0,0],
  #         [0,1,0,1,0,0],
  #         [0,1,0,0,1,0],
  #         [0,0,0,0,1,0]]
  maze_array = []
  maze_array = pd.read_csv("/content/drive/My Drive/Artificial Intelligence/maze_level3.csv",header=None)
  maze_test = np.asarray(maze_array)
  maze_magnified = np.kron(maze_test, np.ones((5,5)))  # Magnify from 80x60 to 800x600

  start = [295,295]#3:[59,59] 2:[250,295] 1:[395,105]
  end = [295,0]#3:[59,0] 2:[345,0] 1:[0,200]
  cost = 1

  dateTimeObj = datetime.now()
  # log file
  log_file = open("/content/drive/My Drive/Artificial Intelligence/level3_logfile.txt", "a")
  log_file.writelines("Start : " + str(dateTimeObj) + "\n")
  log_file.close()

  path = maze_solve(maze_magnified, cost, start, end)

  log_file = open("/content/drive/My Drive/Artificial Intelligence/level3_logfile.txt", "a")
  log_file.writelines("\nSteps:"+ str(path)+"\nEnd : " + str(dateTimeObj) + "\n" )
  log_file.close()
  print(path)